In [28]:
import matplotlib.pyplot as plt 
import matplotlib.image as mpimg 
import numpy as np 
import cv2
%matplotlib inline
import math
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

#function to calculate the x_intercept based on
# the known point, slope and y-line
def get_x_intercept (y1,y2,x2,m):
    x1 = x2 + (y1 - y2)/m
    x = int(x1)
    return x

def draw_lines(img, lines,vertices, color=[255, 0, 0], thickness=10):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """

    #Calculate the x coordinate of the centre of region of interest as per top side
    rcentre = int((vertices[0][2][0] + vertices[0][1][0])/2)
    
    left_line = []
    right_line = []

    #Separate the lines into 2 sets - left lines and right lines based on slope
    #Discard the lines with slope = 0
    #Discard the line with +ve slope that lie in left half
    #Discard the line with -ve slope that line in right half
    for line in lines:
        for x1,y1,x2,y2 in line:
            m=((y2 - y1)/(x2 - x1))
            #print(x1,y1,x2,y2,m)
            if m > 0:
                if (x1 > rcentre) and (x2 > rcentre):
                    right_line.append([x1,y1,x2,y2])
            elif m < 0:
                if (x1 < rcentre) and (x2 < rcentre):
                    left_line.append([x1,y1,x2,y2])
    
    #If any genuine lines are detected then go ahead
    #Average the lines for both sets
    #Extrapolate the average line
    if len(right_line) != 0:
        right_line_arr = np.asarray(right_line)
        rline = np.asarray(right_line_arr.mean(0), dtype=np.int32)
        r_slope = (rline[3] - rline[1])/(rline[2] - rline[0])
        srline = np.zeros((4), dtype=np.int32)
        srline[1] = vertices[0][0][1]
        srline[3] = vertices[0][1][1]
        srline[0] = get_x_intercept (srline[1],rline[1],rline[0],r_slope)
        srline[2] = get_x_intercept (srline[3],rline[1],rline[0],r_slope)
        #cv2.line(img, (rline[0], rline[1]), (rline[2], rline[3]), color, 2)
        cv2.line(img, (srline[0], srline[1]), (srline[2], srline[3]), color, thickness)
        
    if len(left_line) != 0:
        left_line_arr = np.asarray(left_line)
        lline = np.asarray(left_line_arr.mean(0), dtype=np.int32)
        l_slope = (lline[3] - lline[1])/(lline[2] - lline[0])
        slline = np.zeros((4), dtype=np.int32)
        slline[1] = vertices[0][0][1]
        slline[3] = vertices[0][1][1]
        slline[0] = get_x_intercept (slline[1],lline[1],lline[0],l_slope)
        slline[2] = get_x_intercept (slline[3],lline[1],lline[0],l_slope)
        #cv2.line(img, (lline[0], lline[1]), (lline[2], lline[3]), color, 2)
        cv2.line(img, (slline[0], slline[1]), (slline[2], slline[3]), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap, roi_vertices):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines, roi_vertices)
    return line_img
# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

def plot_lane_image(res, gray, edges,roi,line_image,final):
    
    plt.figure(1)
    plt.imshow(gray, cmap='gray')
    plt.figure(2)
    plt.imshow(edges, cmap='Greys_r')
    plt.figure(3)
    plt.imshow(roi, cmap='Greys_r')
    plt.figure(4)
    plt.imshow(line_image, cmap='Greys_r')
    plt.figure(5)
    plt.imshow(final)

def process_image(imagergb):
    
    #Get mask for detecting the pixels with yellow color
    hsv = cv2.cvtColor(imagergb, cv2.COLOR_RGB2HSV)
    lower_yellow = np.array([20, 100, 100])
    upper_yellow = np.array([30, 255, 255])
    y_mask = cv2.inRange(hsv, lower_yellow, upper_yellow)
    
    #Get mask for detecting the pixels with white color
    rgb_threshold = [200,200,200]
    white = (imagergb[:,:,0] > rgb_threshold[0]) & \
                    (imagergb[:,:,1] > rgb_threshold[1]) & \
                    (imagergb[:,:,2] > rgb_threshold[2])        
    w_mask = np.asarray(white, dtype=np.int8)
    
    
    #ORing of both the masks
    c_mask = np.asarray((w_mask | y_mask), dtype=np.int8)
    
    # Bitwise-AND mask and original image
    masked = cv2.bitwise_and(imagergb,imagergb, mask=c_mask)
    
    #Convert the image to grayscale
    gray = grayscale (masked)
    
    #Apply gaussian filter
    kernel_size = 3 # Must be an odd number (3, 5, 7...)
    blur_gray = gaussian_blur(gray, kernel_size)
    
    #Apply canny edge detection
    low_threshold = 100
    high_threshold = 150    
    edges = canny(blur_gray, low_threshold, high_threshold)

    #Extract the region of interest
    #Calculate the span of the vertices with respect to the resolution
    imshape = imagergb.shape
    x_bottom_left = (130/960)
    x_top_left = (440/960)
    y_top = (330/540)
    x_top_right = (520/960)
    x_bottom_right = (900/960)
    
    vertices = np.array([[(int(x_bottom_left*imshape[1]),imshape[0]),\
                          (int(x_top_left*imshape[1]),int(y_top*imshape[0])),\
                          (int(x_top_right*imshape[1]),int(y_top*imshape[0])),\
                          (int(x_bottom_right*imshape[1]),imshape[0])]], dtype=np.int32)
    
    roi = region_of_interest(edges, vertices)
    
    #Detect lines that represent the left and Right lane
    rho = 1
    theta = np.pi/180
    threshold = 20
    min_line_len =20
    max_line_gap =20
    line_image = hough_lines(roi, rho, theta, threshold, min_line_len, max_line_gap, vertices)
    
    #Superimpose the annotated lines on the original image
    final = weighted_img(line_image, imagergb, 0.8, 1., λ=0.)
    #plot_lane_image(masked,gray, edges,roi,line_image,final)
    return final


#Get output for solidWhiteRight.mp4
white_output = 'white.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

#Get output for solidYellowLeft.mp4
yellow_output = 'yellow.mp4'
clip1 = VideoFileClip("solidYellowLeft.mp4")
yellow_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time yellow_clip.write_videofile(yellow_output, audio=False)

#Get output for challenge.mp4
output = 'ch_output.mp4'
clip1 = VideoFileClip("challenge.mp4")
ch_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time ch_clip.write_videofile(output, audio=False)



[MoviePy] >>>> Building video white.mp4
[MoviePy] Writing video white.mp4


100%|███████████████████████████████████████▊| 221/222 [00:06<00:00, 35.02it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: white.mp4 

Wall time: 7.49 s
[MoviePy] >>>> Building video yellow.mp4
[MoviePy] Writing video yellow.mp4


100%|███████████████████████████████████████▉| 681/682 [00:21<00:00, 31.95it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: yellow.mp4 

Wall time: 22.4 s
[MoviePy] >>>> Building video ch_output.mp4
[MoviePy] Writing video ch_output.mp4


100%|████████████████████████████████████████| 251/251 [00:14<00:00, 16.40it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ch_output.mp4 

Wall time: 16.3 s
